In [1]:
# === Development Environment ===
"""
Python version:       3.10.15 
pandas version:       2.2.3
numpy version:        1.26.4
scikit-learn version: 1.5.1
matplotlib version: 3.9.1
"""

'\nPython version:       3.10.15 \npandas version:       2.2.3\nnumpy version:        1.26.4\nscikit-learn version: 1.5.1\nmatplotlib version: 3.9.1\n'

In [5]:
import pandas as pd
import sys
import numpy as np
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc, f1_score, precision_score, accuracy_score
from sklearn.utils import resample

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import pi
from sklearn.metrics import average_precision_score

In [6]:
mRNA_result = pd.read_csv("../analysis/mRNA_pred.csv")
mRNA_result

,Peptide,MHC,MMS_Cluster,Label,ImmunoMTL_score,pMHC,BigMHC_IM,PRIME_score,PRIME_rank,munis,netMHCpan_score,netMHCpan_rank,patientID
0,ITVNASRPQPF,HLA-A*32:01,0.0,0,9.296952e-01,ITVNASRPQPFHLA-A*32:01,0.000514,0.002020,19.170,0.151489,0.0126,4.6814,1
1,RLAAAVRF,HLA-A*32:01,0.0,0,2.207086e-08,RLAAAVRFHLA-A*32:01,0.006755,0.004967,8.816,0.271729,0.0734,1.5100,5
2,AVMHLDHSDTI,HLA-A*32:01,0.0,0,4.035173e-01,AVMHLDHSDTIHLA-A*32:01,0.000256,0.014185,3.145,0.095215,0.0201,3.6059,5
3,VSYANSCANPV,HLA-C*17:01,0.0,0,3.250802e-07,VSYANSCANPVHLA-C*17:01,0.028055,0.000734,37.307,0.005650,0.0019,14.5042,1
4,MSAEVNLAGL,HLA-C*17:01,0.0,1,4.604495e-02,MSAEVNLAGLHLA-C*17:01,0.100420,0.018708,2.849,0.228760,0.1320,1.5382,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,FFPYNYHSQK,HLA-C*04:01,3.0,0,1.400116e-03,FFPYNYHSQKHLA-C*04:01,0.331144,0.011502,3.973,0.008041,0.0012,12.7056,14
218,TEYKLVVVGAC,HLA-C*04:01,3.0,0,9.890347e-01,TEYKLVVVGACHLA-C*04:01,0.035441,0.000607,41.408,0.005188,0.0000,61.6667,20
219,SFASFQRDLF,HLA-C*04:01,3.0,0,7.486498e-12,SFASFQRDLFHLA-C*04:01,0.140121,0.011137,4.101,0.176514,0.0053,5.4123,20
220,SSDGNLNEL,HLA-C*08:02,3.0,0,1.820668e-01,SSDGNLNELHLA-C*08:02,0.276676,0.284666,0.003,0.997559,0.9733,0.0079,29


In [7]:
pos_mRNA_result = mRNA_result[mRNA_result["Label"]==1]
pos_mRNA_result.groupby("patientID").count()

,Peptide,MHC,MMS_Cluster,Label,ImmunoMTL_score,pMHC,BigMHC_IM,PRIME_score,PRIME_rank,munis,netMHCpan_score,netMHCpan_rank
patientID,,,,,,,,,,,,
5,1,1,1,1,1,1,1,1,1,1,1,1
6,1,1,1,1,1,1,1,1,1,1,1,1
10,7,7,7,7,7,7,7,7,7,7,7,7
11,3,3,3,3,3,3,3,3,3,3,3,3
14,1,1,1,1,1,1,1,1,1,1,1,1
25,3,3,3,3,3,3,3,3,3,3,3,3
29,5,5,5,5,5,5,5,5,5,5,5,5


In [8]:
p10 = mRNA_result[mRNA_result["patientID"] == 10]
p11 = mRNA_result[mRNA_result["patientID"] == 11]
p25 = mRNA_result[mRNA_result["patientID"] == 25]
p29 = mRNA_result[mRNA_result["patientID"] == 29]

In [9]:
#accumulate prism
import matplotlib.pyplot as plt
prediction_columns = ['ImmunoMTL_score', 'BigMHC_IM', 'munis', 'PRIME_rank', 'netMHCpan_rank']
precision_at_k = {column: [] for column in prediction_columns}
avg_top_k = {}

def calculate_accumulated_precision(df, prediction_columns, k):
    #if prediction_columns == 'IMMpred':
        #df = df[df["EL_rank"]<2]
    #k = len(df[df["IMMpred"] >= 0.5])
    #print(k)
    precision_at_k_df = pd.DataFrame(index=range(1, k+1))
    acc_at_k_df = pd.DataFrame(index=range(1, k+1))
    tp_at_k_df = pd.DataFrame(index=range(1, k+1))

    for column in prediction_columns:
        # Sort the dataframe based on the model's predictions
        # Note: We need to sort ranks in ascending order, but scores in descending order
        aaaascending = True if 'rank' in column.lower() else False
        sorted_df = df.sort_values(by=column, ascending=aaaascending)

        # Calculate the accumulated precision at each k
        accumulated_precision = []
        true_positives = 0
        top_k_scores = []
        for i in range(k):
            if sorted_df.iloc[i]['Label'] == 1:
                true_positives += 1
            accumulated_precision.append(true_positives / (i+1))
            top_k_scores.append(true_positives / (i+1))
            
        precision_at_k_df[column] = accumulated_precision
        # Calculate and store median of top k predictions
        avg_top_k[column] = np.mean(top_k_scores)
        
        accumulated_acc = []
        true_positive_negative = 0
        for i in range(k):
            if sorted_df.iloc[i]['Label'] == sorted_df.iloc[i][column]:
                true_positive_negative += 1
            accuracy_at_k = true_positive_negative / (i+1)
            accumulated_acc.append(accuracy_at_k)
        acc_at_k_df[column] = accumulated_precision
        
        accumulated_tp = []
        true_positive = 0
        for i in range(k):
            if sorted_df.iloc[i]['Label'] == 1:
                true_positive += 1
            accumulated_tp.append(true_positive)
        tp_at_k_df[column] = accumulated_tp

    return precision_at_k_df, acc_at_k_df, avg_top_k

In [10]:
def count_peptides_until_first_positive(df, prediction_columns):
    first_pos_index = {}

    for column in prediction_columns:
        # Sort predictions: lower is better for ranks, higher is better for scores
        ascending = True if 'rank' in column.lower() else False
        sorted_df = df.sort_values(by=column, ascending=ascending).reset_index(drop=True)

        # Find the index of the first positive label (Label == 1)
        first_positive_idx = sorted_df[sorted_df['Label'] == 1].index.min()

        # If there's no positive, return NaN
        if pd.isna(first_positive_idx):
            first_pos_index[column] = np.nan
        else:
            # Add 1 to make it human-readable (1-based index)
            first_pos_index[column] = first_positive_idx + 1

    return first_pos_index

In [11]:
# Number of top predictions to calculate precision for
top_k = 10

# Calculate the accumulated precision for each model up to top_k
precision_at_k_results, acc_at_k_results, avg_top_k = calculate_accumulated_precision(p10, prediction_columns, 10)
first_pos_index = count_peptides_until_first_positive(p10, prediction_columns)
print(precision_at_k_results)
print(avg_top_k)
print(acc_at_k_results)
print(first_pos_index)
# Output the results to a CSV file
#precision_at_k_results.to_csv('../analysis/p29_acc.csv', index_label='Top k predictions')
precision_at_k_results.to_csv("../analysis/mRNA_p10.csv", index = False)

    ImmunoMTL_score  BigMHC_IM     munis  PRIME_rank  netMHCpan_rank
1          1.000000   0.000000  1.000000    1.000000        1.000000
2          1.000000   0.500000  1.000000    1.000000        1.000000
3          0.666667   0.333333  0.666667    1.000000        1.000000
4          0.750000   0.500000  0.750000    1.000000        1.000000
5          0.800000   0.600000  0.600000    0.800000        1.000000
6          0.833333   0.500000  0.666667    0.833333        0.833333
7          0.857143   0.571429  0.714286    0.714286        0.714286
8          0.750000   0.625000  0.750000    0.750000        0.625000
9          0.666667   0.555556  0.666667    0.777778        0.555556
10         0.600000   0.600000  0.600000    0.700000        0.600000
{'ImmunoMTL_score': 0.7923809523809523, 'BigMHC_IM': 0.478531746031746, 'munis': 0.7414285714285713, 'PRIME_rank': 0.8575396825396826, 'netMHCpan_rank': 0.8328174603174603}
    ImmunoMTL_score  BigMHC_IM     munis  PRIME_rank  netMHCpan_rank

In [12]:
# Number of top predictions to calculate precision for
top_k = 10

# Calculate the accumulated precision for each model up to top_k
precision_at_k_results, acc_at_k_results, avg_top_k = calculate_accumulated_precision(p11, prediction_columns, 10)
first_pos_index = count_peptides_until_first_positive(p11, prediction_columns)

print(precision_at_k_results)
print(avg_top_k)
print(acc_at_k_results)
print(first_pos_index)

# Output the results to a CSV file
precision_at_k_results.to_csv("../analysis/mRNA_p11.csv", index = False)

    ImmunoMTL_score  BigMHC_IM     munis  PRIME_rank  netMHCpan_rank
1          0.000000   0.000000  0.000000    0.000000        0.000000
2          0.000000   0.500000  0.000000    0.000000        0.000000
3          0.333333   0.333333  0.000000    0.333333        0.000000
4          0.500000   0.250000  0.250000    0.250000        0.000000
5          0.400000   0.200000  0.200000    0.200000        0.000000
6          0.333333   0.166667  0.166667    0.166667        0.166667
7          0.285714   0.142857  0.142857    0.142857        0.142857
8          0.250000   0.250000  0.125000    0.125000        0.125000
9          0.222222   0.222222  0.111111    0.111111        0.111111
10         0.200000   0.200000  0.200000    0.200000        0.200000
{'ImmunoMTL_score': 0.25246031746031744, 'BigMHC_IM': 0.2265079365079365, 'munis': 0.11956349206349207, 'PRIME_rank': 0.1528968253968254, 'netMHCpan_rank': 0.07456349206349207}
    ImmunoMTL_score  BigMHC_IM     munis  PRIME_rank  netMHCpan_

In [13]:
# Number of top predictions to calculate precision for
top_k = 10

# Calculate the accumulated precision for each model up to top_k
precision_at_k_results, acc_at_k_results, avg_top_k = calculate_accumulated_precision(p25, prediction_columns, 10)
first_pos_index = count_peptides_until_first_positive(p25, prediction_columns)

print(precision_at_k_results)
print(avg_top_k)
print(acc_at_k_results)
print(first_pos_index)

# Output the results to a CSV file
#precision_at_k_results.to_csv('../analysis/p29_acc.csv', index_label='Top k predictions')
precision_at_k_results.to_csv("../analysis/mRNA_p25.csv", index = False)

    ImmunoMTL_score  BigMHC_IM     munis  PRIME_rank  netMHCpan_rank
1          1.000000   0.000000  0.000000    0.000000        0.000000
2          1.000000   0.000000  0.000000    0.500000        0.000000
3          0.666667   0.000000  0.000000    0.333333        0.333333
4          0.500000   0.250000  0.000000    0.250000        0.250000
5          0.400000   0.200000  0.200000    0.200000        0.200000
6          0.333333   0.166667  0.166667    0.166667        0.166667
7          0.285714   0.142857  0.142857    0.142857        0.142857
8          0.250000   0.125000  0.125000    0.125000        0.125000
9          0.222222   0.222222  0.111111    0.111111        0.111111
10         0.200000   0.200000  0.100000    0.200000        0.200000
{'ImmunoMTL_score': 0.4857936507936508, 'BigMHC_IM': 0.13067460317460317, 'munis': 0.08456349206349206, 'PRIME_rank': 0.20289682539682538, 'netMHCpan_rank': 0.1528968253968254}
    ImmunoMTL_score  BigMHC_IM     munis  PRIME_rank  netMHCpan_

In [14]:
# Number of top predictions to calculate precision for
top_k = 10

# Calculate the accumulated precision for each model up to top_k
precision_at_k_results, acc_at_k_results, avg_top_k = calculate_accumulated_precision(p29, prediction_columns, 10)
first_pos_index = count_peptides_until_first_positive(p29, prediction_columns)

print(precision_at_k_results)
print(avg_top_k)
print(acc_at_k_results)
print(first_pos_index)

# Output the results to a CSV file
#precision_at_k_results.to_csv('../analysis/p29_acc.csv', index_label='Top k predictions')
precision_at_k_results.to_csv("../analysis/mRNA_p29.csv", index = False)

    ImmunoMTL_score  BigMHC_IM     munis  PRIME_rank  netMHCpan_rank
1          1.000000   0.000000  0.000000    0.000000        0.000000
2          0.500000   0.000000  0.500000    0.500000        0.500000
3          0.333333   0.333333  0.333333    0.333333        0.333333
4          0.250000   0.250000  0.250000    0.250000        0.250000
5          0.200000   0.400000  0.400000    0.200000        0.200000
6          0.333333   0.333333  0.500000    0.166667        0.333333
7          0.285714   0.285714  0.428571    0.142857        0.285714
8          0.250000   0.250000  0.375000    0.125000        0.375000
9          0.222222   0.333333  0.333333    0.111111        0.444444
10         0.200000   0.400000  0.400000    0.200000        0.500000
{'ImmunoMTL_score': 0.3574603174603174, 'BigMHC_IM': 0.25857142857142856, 'munis': 0.35202380952380957, 'PRIME_rank': 0.20289682539682538, 'netMHCpan_rank': 0.3221825396825396}
    ImmunoMTL_score  BigMHC_IM     munis  PRIME_rank  netMHCpan_